# 1.CNN_Block.ipynb 說明
本 notebook 主要教學如何用 PyTorch 建立卷積神經網路（CNN）的基本區塊，並進行簡單測試。內容重點如下：
1. **安裝與匯入套件**：安裝 torchinfo（用於顯示模型結構），匯入 PyTorch 相關模組與 summary 工具。
2. **裝置選擇**：自動偵測是否有 GPU（cuda），有則用 GPU，否則用 CPU。
3. **啟動函數工具**：定義 activation_func，可根據字串選擇不同 activation function（如 relu、leaky_relu、sigmoid、prelu、softmax、gelu）。
4. **CNNBlock 類別**：自訂 CNNBlock 類別，包含卷積層、BatchNorm 層、啟動函數，並定義 forward 方法（卷積→啟動→正規化）。
5. **CNNBlock 測試**：建立隨機輸入張量，實例化 CNNBlock，將輸入丟進模型並印出輸出 shape。
6. **更完整的 CNN 模型結構**：定義 ModelStructure 類別，堆疊多個 CNNBlock 與池化層，最後接全連接層（FC），可用於分類任務。
7. **模型結構摘要**：用 summary 工具顯示整個模型的結構、每層的輸入/輸出 shape 及參數數量。
本 notebook 讓你熟悉如何用 PyTorch 自訂 CNN 卷積區塊、組合成完整模型，並進行 forward 測試與結構檢查，是 CNN 架構設計的基礎教學範例。

In [3]:
# 安裝 torchinfo 套件，用於顯示 PyTorch 模型的結構與參數資訊
!pip install torchinfo

In [4]:
# 匯入必要的套件
import torch.nn as nn  # 匯入 PyTorch 的神經網路模組，方便建立神經網路結構
import torch           # 匯入 PyTorch 主套件，進行張量運算與 GPU 加速
from torchinfo import summary  # 匯入 summary 函數，用於顯示模型結構摘要

In [5]:
# 選擇運算裝置 (GPU 或 CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # 若有 GPU 則使用 GPU，否則使用 CPU
device  # 顯示目前使用的裝置


device(type='cpu')

In [6]:
# 匯入必要的套件
import torch.nn as nn

# 定義一個函數，根據傳入的 activation 字串回傳對應的啟動函數 (Activation Function) 物件
def activation_func(activation):
    return nn.ModuleDict({
        'relu': nn.ReLU(inplace=True),   # ReLU 激活函數，inplace=True 代表直接在原 tensor 上修改，節省記憶體
        'leaky_relu': nn.LeakyReLU(negative_slope=0.01, inplace=True),  # LeakyReLU，負值部分有微小斜率
        'sigmoid': nn.Sigmoid(),         # Sigmoid 激活函數，常用於二元分類輸出
        'prelu': nn.PReLU(),             # PReLU，負值部分的斜率可學習
        'softmax': nn.Softmax(dim=1),    # Softmax，常用於多分類輸出，dim=1 代表對每個樣本的 channel 做 softmax
        'gelu': nn.GELU()                # GELU，常用於較新型的神經網路架構
    })[activation]  # 根據傳入的 activation 字串回傳對應的物件

In [7]:
import torch.nn as nn

# 定義一個 CNNBlock 類別，繼承自 nn.Module，作為卷積神經網路的基本區塊
class CNNBlock(nn.Module):
    def __init__(self, in_channel, out_channel, k_size, activation='relu', pad=1, s=1, dilation=1):
        super().__init__()
        # 定義 2D 卷積層
        self.conv = nn.Conv2d(in_channel, out_channel,
                              k_size, padding=pad, stride=s, dilation=dilation) # Group_channel
        # 定義 Batch Normalization 層，幫助加速訓練與穩定收斂
        self.batchNorm = nn.BatchNorm2d(out_channel)
        # 定義啟動函數 (Activation Function)
        self.actfunction = activation_func(activation)

    def forward(self, x):
        # 前向傳播流程：先卷積 -> 啟動函數 -> 批次正規化
        x = self.conv(x)           # 卷積運算
        x = self.actfunction(x)    # 啟動函數
        x = self.batchNorm(x)      # 批次正規化
        return x
# bn 跟 activation function能不能換位置
# forward這個名字能不能換掉

In [8]:
# 測試 CNNBlock 卷積區塊
input = torch.randn(1, 64, 3, 3) # 建立一個隨機輸入張量，形狀為 (batch=1, channel=64, 高=3, 寬=3)
model = CNNBlock(64, 128, 3)     # 建立一個 CNNBlock，輸入通道 64，輸出通道 128，卷積核大小 3x3
# summary(model, input_data=input) # 可以用 summary 查看模型結構 (此行目前註解掉)
output = model(input)             # 將輸入資料傳入模型，取得輸出
print(output.size())              # 印出輸出張量的形狀

torch.Size([1, 128, 3, 3])


In [9]:
# 定義一個更完整的 CNN 模型結構
class ModelStructure(nn.Module):
    def __init__(self, cls):
        super().__init__()
        # 第一層卷積區塊，輸入 3 通道，輸出 6 通道，卷積核 5x5，無 padding
        self.conv1 = CNNBlock(in_channel = 3, out_channel = 6, k_size = 5, activation='relu', pad = 0)
        # 第二層卷積區塊，輸入 6 通道，輸出 16 通道，卷積核 5x5，無 padding
        self.conv2 = CNNBlock(in_channel = 6, out_channel = 16, k_size = 5, activation='relu', pad = 0)
        # 第三層卷積區塊，輸入 16 通道，輸出 120 通道，卷積核 3x3，無 padding
        self.conv3 = CNNBlock(in_channel = 16, out_channel = 120, k_size = 3, activation='relu', pad = 0)

        # 定義最大池化層，每次將特徵圖縮小一半
        self.pool = nn.MaxPool2d(2, 2)
        # 定義 ReLU 啟動函數
        self.act = nn.ReLU(True)

        # 定義全連接層 (Fully Connected Layer)
        self.fc = nn.Sequential(
            nn.Linear(in_features = 120*25*25, out_features = 84),  # 第一層全連接，輸入 120*25*25，輸出 84
            nn.ReLU(True),
            nn.Linear(in_features = 84, out_features = cls),         # 第二層全連接，輸出為分類數量
        )

    def forward(self, x):
        # 前向傳播流程：卷積區塊 + 啟動函數 + 池化，重複三次
        x = self.pool(self.act(self.conv1(x)))
        x = self.pool(self.act(self.conv2(x)))
        x = self.pool(self.act(self.conv3(x)))
        # print(x.size())  # 可用於檢查特徵圖尺寸
        x = x.view(-1, 120*25*25) # 將多維特徵圖展平成一維向量 (也可用 .flatten())
        x = self.fc(x)            # 通過全連接層
        return x

# 建立模型，分類數量設為 2，並移至指定裝置 (GPU/CPU)
cnn_model = ModelStructure(cls=2).to(device)

# 建立測試用的隨機張量，形狀為 (batch=1, channel=3, 高=224, 寬=224)，並移至裝置
test_tensor = torch.randn((1, 3, 224, 224)).to(device)
output = cnn_model(test_tensor)  # 將測試資料傳入模型，取得輸出
print(output.size())             # 印出輸出張量的形狀

# 25*25 ?  # 註解：經過三次池化後，原本 224x224 會縮小為 25x25
# nn.ReLU(True)?  # 註解：inplace=True 代表直接在原 tensor 上修改
# why .view()不會多存一份copy()? reshape()會存一份copy?  # 註解：.view() 只改變形狀，不複製資料

torch.Size([1, 2])


In [10]:
# 顯示模型結構摘要，包含每層的輸入/輸出形狀與參數數量
summary(cnn_model, input_data=test_tensor)

Layer (type:depth-idx)                   Output Shape              Param #
ModelStructure                           [1, 2]                    --
├─CNNBlock: 1-1                          [1, 6, 220, 220]          --
│    └─Conv2d: 2-1                       [1, 6, 220, 220]          456
│    └─ReLU: 2-2                         [1, 6, 220, 220]          --
│    └─BatchNorm2d: 2-3                  [1, 6, 220, 220]          12
├─ReLU: 1-2                              [1, 6, 220, 220]          --
├─MaxPool2d: 1-3                         [1, 6, 110, 110]          --
├─CNNBlock: 1-4                          [1, 16, 106, 106]         --
│    └─Conv2d: 2-4                       [1, 16, 106, 106]         2,416
│    └─ReLU: 2-5                         [1, 16, 106, 106]         --
│    └─BatchNorm2d: 2-6                  [1, 16, 106, 106]         32
├─ReLU: 1-5                              [1, 16, 106, 106]         --
├─MaxPool2d: 1-6                         [1, 16, 53, 53]           --
├─CNNBlock: